In [1]:
import pandas as pd

def get_new_feature_name_df(old_feature_name_df):
    feature_dup_df = pd.DataFrame(data=old_feature_name_df.groupby('column_name').cumcount(),
                                  columns=['dup_cnt'])
    feature_dup_df = feature_dup_df.reset_index()
    new_feature_name_df = pd.merge(old_feature_name_df.reset_index(), feature_dup_df, how='outer')
    new_feature_name_df['column_name'] = new_feature_name_df[['column_name', 'dup_cnt']].apply(lambda x : x[0]+'_'+str(x[1]) 
                                                                                         if x[1] >0 else x[0] ,  axis=1)
    new_feature_name_df = new_feature_name_df.drop(['index'], axis=1)
    return new_feature_name_df

def get_human_dataset( ):
    
    # 각 데이터 파일들은 공백으로 분리되어 있으므로 read_csv에서 공백 문자를 sep으로 할당.
    feature_name_df = pd.read_csv(r'C:\Users\takea\Desktop\DeepLearning\datasets\week3\HAR_Dataset\UCI_HAR_Dataset\features.txt',sep='\s+',
                        header=None,names=['column_index','column_name'])
    
    # 중복된 피처명을 수정하는 get_new_feature_name_df()를 이용, 신규 피처명 DataFrame생성. 
    new_feature_name_df = get_new_feature_name_df(feature_name_df)
    
    # DataFrame에 피처명을 컬럼으로 부여하기 위해 리스트 객체로 다시 변환
    feature_name = new_feature_name_df.iloc[:, 1].values.tolist()
    
    # 학습 피처 데이터 셋과 테스트 피처 데이터을 DataFrame으로 로딩. 컬럼명은 feature_name 적용
    X_train = pd.read_csv(r'C:\Users\takea\Desktop\DeepLearning\datasets\week3\HAR_Dataset\UCI_HAR_Dataset\train\X_train.txt',sep='\s+', names=feature_name )
    X_test = pd.read_csv(r'C:\Users\takea\Desktop\DeepLearning\datasets\week3\HAR_Dataset\UCI_HAR_Dataset\test\X_test.txt',sep='\s+', names=feature_name)
    
    # 학습 레이블과 테스트 레이블 데이터을 DataFrame으로 로딩하고 컬럼명은 action으로 부여
    y_train = pd.read_csv(r'C:\Users\takea\Desktop\DeepLearning\datasets\week3\HAR_Dataset\UCI_HAR_Dataset\train\y_train.txt',sep='\s+',header=None,names=['action'])
    y_test = pd.read_csv(r'C:\Users\takea\Desktop\DeepLearning\datasets\week3\HAR_Dataset\UCI_HAR_Dataset\test\y_test.txt',sep='\s+',header=None,names=['action'])
    
    # 로드된 학습/테스트용 DataFrame을 모두 반환 
    return X_train, X_test, y_train, y_test

In [2]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

# 결정 트리에서 사용한 get_human_dataset( )을 이용해 학습/테스트용 DataFrame 반환
X_train, X_test, y_train, y_test = get_human_dataset()

print("## 학습 피처 데이터 정보 ##")
print(X_train.info())

C:\Users\takea\anaconda3\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


## 학습 피처 데이터 정보 ##
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7352 entries, 0 to 7351
Columns: 561 entries, tBodyAcc-mean()-X to angle(Z,gravityMean)
dtypes: float64(561)
memory usage: 31.5 MB
None


In [3]:
print(X_train)

      tBodyAcc-mean()-X  tBodyAcc-mean()-Y  tBodyAcc-mean()-Z  \
0              0.288585          -0.020294          -0.132905   
1              0.278419          -0.016411          -0.123520   
2              0.279653          -0.019467          -0.113462   
3              0.279174          -0.026201          -0.123283   
4              0.276629          -0.016570          -0.115362   
...                 ...                ...                ...   
7347           0.299665          -0.057193          -0.181233   
7348           0.273853          -0.007749          -0.147468   
7349           0.273387          -0.017011          -0.045022   
7350           0.289654          -0.018843          -0.158281   
7351           0.351503          -0.012423          -0.203867   

      tBodyAcc-std()-X  tBodyAcc-std()-Y  tBodyAcc-std()-Z  tBodyAcc-mad()-X  \
0            -0.995279         -0.983111         -0.913526         -0.995112   
1            -0.998245         -0.975300         -0.960322 

In [4]:
print(y_train)

      action
0          5
1          5
2          5
3          5
4          5
...      ...
7347       2
7348       2
7349       2
7350       2
7351       2

[7352 rows x 1 columns]


In [5]:
unique_labels = y_train.action.drop_duplicates()
print(unique_labels)

0      5
27     4
51     6
78     1
125    3
150    2
Name: action, dtype: int64


# BiLSTM

In [6]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Bidirectional, LSTM, Dense, Dropout
from tensorflow.keras.utils import to_categorical

In [7]:
# 데이터 스케일링
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# 레이블을 원-핫 인코딩
y_train = to_categorical(y_train - 1, num_classes=6)
y_test = to_categorical(y_test - 1, num_classes=6)

In [ ]:
# BiLSTM 모델 정의
model = Sequential()
model.add(Bidirectional(LSTM(64, return_sequences=True), input_shape=(X_train.shape[1], 1)))
model.add(Bidirectional(LSTM(64)))
model.add(Dropout(0.5))
model.add(Dense(6, activation='softmax'))

# 모델 컴파일
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# 모델 훈련
history = model.fit(X_train.reshape(-1, X_train.shape[1], 1), y_train, epochs=20, batch_size=64, validation_split=0.2)

Epoch 1/20
92/92 [==============================] - 227s 2s/step - loss: 1.0466 - accuracy: 0.5113 - val_loss: 0.7842 - val_accuracy: 0.6893
Epoch 2/20
92/92 [==============================] - 235s 3s/step - loss: 0.6624 - accuracy: 0.7169 - val_loss: 0.7251 - val_accuracy: 0.7417
Epoch 3/20
92/92 [==============================] - 257s 3s/step - loss: 0.5742 - accuracy: 0.7577 - val_loss: 0.5926 - val_accuracy: 0.8097
Epoch 4/20
92/92 [==============================] - 469s 5s/step - loss: 0.5021 - accuracy: 0.7903 - val_loss: 0.5538 - val_accuracy: 0.8389
Epoch 5/20
92/92 [==============================] - 556s 6s/step - loss: 0.4492 - accuracy: 0.8119 - val_loss: 0.5392 - val_accuracy: 0.8253
Epoch 6/20
92/92 [==============================] - 602s 7s/step - loss: 0.4265 - accuracy: 0.8181 - val_loss: 0.5483 - val_accuracy: 0.8457
Epoch 7/20
92/92 [==============================] - 629s 7s/step - loss: 0.3916 - accuracy: 0.8374 - val_loss: 0.4689 - val_accuracy: 0.8511
Epoch 8/20
92

In [ ]:
# 정확도 그래프
plt.plot(history.history['accuracy'], label='Train Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

# 손실 그래프
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

# 테스트 데이터로 평가
test_loss, test_acc = model.evaluate(X_test.reshape(-1, X_test.shape[1], 1), y_test)
print(f'Test accuracy: {test_acc}')


# GRU

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import GRU, Dense, Dropout
from tensorflow.keras.utils import to_categorical

In [ ]:
# 데이터 스케일링
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# 레이블을 원-핫 인코딩
y_train = to_categorical(y_train - 1, num_classes=6)
y_test = to_categorical(y_test - 1, num_classes=6)

In [ ]:
# GRU 모델 정의
model = Sequential()
model.add(GRU(64, return_sequences=True, input_shape=(X_train.shape[1], 1)))
model.add(GRU(64))
model.add(Dropout(0.5))
model.add(Dense(6, activation='softmax'))

# 모델 컴파일
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# 모델 훈련
history = model.fit(X_train.reshape(-1, X_train.shape[1], 1), y_train, epochs=20, batch_size=64, validation_split=0.2)

In [ ]:
# 정확도 그래프
plt.plot(history.history['accuracy'], label='Train Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

# 손실 그래프
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

# 테스트 데이터로 평가
test_loss, test_acc = model.evaluate(X_test.reshape(-1, X_test.shape[1], 1), y_test)
print(f'Test accuracy: {test_acc}')

# LinearSVM

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.svm import LinearSVC
from sklearn.metrics import classification_report, confusion_matrix

In [ ]:
# 데이터 스케일링
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [ ]:
# Linear SVM 모델 정의
svm = LinearSVC()

# 모델 훈련
svm.fit(X_train, y_train)

In [ ]:
# 예측
y_pred = svm.predict(X_test)

# 분류 보고서 출력
print(classification_report(y_test, y_pred))

# 혼동 행렬 시각화
cm = confusion_matrix(y_test, y_pred)
plt.imshow(cm, interpolation='nearest', cmap=plt.cm.Blues)
plt.title('Confusion Matrix')
plt.colorbar()
plt.xlabel('Predicted')
plt.ylabel('True')
plt.show()